In [20]:
import pandas as pd
from ortools.linear_solver import pywraplp

In [21]:
solver = pywraplp.Solver.CreateSolver('GLOP')

In [22]:
df = pd.read_csv('RA.csv')
df.head()

,Região administrativa,Num de veiculos,% de veiculos,Aluguel medio,Dist centro,lat,lon
0,Plano Piloto,74038,86.9,2268.0,0,-15.797422,-47.878090
1,Gama,25485,65.0,965.0,30,-16.018858,-48.067951
2,Taguatinga,45951,72.0,1037.0,19,-15.803228,-48.017552
3,Brazlândia,8798,56.1,850.0,59,-15.622604,-48.134750
4,Sobradinho,13364,72.9,1003.0,22,-15.633557,-47.829898


## Variáveis do problema

In [23]:
N = len(df)                             # Número de RAs
A = df['Aluguel medio'].to_numpy()      # Aluguel medio
V = df['Num de veiculos'].to_numpy()    # Número de veículos
D = df['Dist centro'].to_numpy()        # Distância entre a RA e o centro de distribuição das vacinas (plano piloto)

cities = df['Região administrativa'].to_numpy()

In [24]:
cap_max = 25000         # Capacidade max de atendimento de cada drive thru
gas = 6/12              # Preço do km rodado (preço da gasolina / consumo)
doses = 500000          # Número de doses de vacina disponíveis
qtde_pessoas = 3        # Quantidade média de pessoas vacinadas por carro

## Variável alvo

Quantidade de drive thrus por Região Administrativa do DF

- **Limite mínimo**: 0
- **Limite máximo**: 2

In [25]:
X = [solver.IntVar(0, 2, f'X{i}') for i in range(N)]

## Restrições

1. Todas as doses disponíveis devem ser distribuídas.

O número de postos drive thru criados deve ser igual ao número de doses disponíveis dividido pela capacidade de atendimento de cada posto


$$ \sum_{i=0}^{N} X_{i}  = \frac{d}{c}$$

In [26]:
ct = solver.Constraint(doses/cap_max, doses/cap_max, 'ct_num_doses')
for i in range(N):
    ct.SetCoefficient(X[i], 1)

2. Uma região só receberá um drive thru se a quantidade de vacinantes (Qtde de carros [V] * média de vacinantes por carro [q]) for maior do que a capacidade de vacinação do posto

$$\sum X_{i} \frac{c}{qV_{i}} \leq 1$$

In [27]:
# ct = solver.Constraint(0, qtde_pessoas/cap_max, 'ct_num_vacinantes')
# for i in range(N):
#     ct.SetCoefficient(X[i], 1/V[i])

## Função objetivo

Minimizar o custo de distribuição das vacinas

Aqui, é considerado o custo de transporte das vacinas até o posto (considerando que o centro de distribuição está localizado no Plano Piloto) e o custo de armazenamento das vacinas (considerando o preço do aluguel médio de cada região administrativa)

$$min \left ( \sum_{i=0}^{N} X_{i} (A_{i} + gD_{i}) \right )$$

In [28]:
objective = solver.Objective()

for i in range(N):
    objective.SetCoefficient(X[i], A[i] + D[i]*gas)
objective.SetMinimization()

## Resultado

In [29]:
status = solver.Solve()

In [30]:
if status == pywraplp.Solver.OPTIMAL:
    print('Valor objetivo =', solver.Objective().Value())
    for i in range(N):
        n = X[i].solution_value()
        if n:
            print(f"{cities[i]}: {n} postos")
else:
    print('O problema não possui solução ótima')

O problema não possui solução ótima
